<a href="https://colab.research.google.com/github/PiotrusWatson/level4project/blob/master/data/ipynbs/textual_entailment_snli.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

lets get the snli dataset baybee

In [1]:
!wget https://nlp.stanford.edu/projects/snli/snli_1.0.zip
!unzip snli_1.0.zip

--2019-10-28 12:34:58--  https://nlp.stanford.edu/projects/snli/snli_1.0.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 94550081 (90M) [application/zip]
Saving to: ‘snli_1.0.zip’

snli_1.0.zip        100%[===================>]  90.17M  26.5MB/s    in 3.6s    

2019-10-28 12:35:02 (25.0 MB/s) - ‘snli_1.0.zip’ saved [94550081/94550081]

Archive:  snli_1.0.zip
   creating: snli_1.0/
  inflating: snli_1.0/.DS_Store      
   creating: __MACOSX/
   creating: __MACOSX/snli_1.0/
  inflating: __MACOSX/snli_1.0/._.DS_Store  
 extracting: snli_1.0/Icon           
  inflating: __MACOSX/snli_1.0/._Icon  
  inflating: snli_1.0/README.txt     
  inflating: __MACOSX/snli_1.0/._README.txt  
  inflating: snli_1.0/snli_1.0_dev.jsonl  
  inflating: snli_1.0/snli_1.0_dev.txt  
  inflating: snli_1.0/snli_1.0_test.jsonl  
  inflating: snli_1.0/snli_1.0_

In [2]:
# Download the Glove.zip file and expand it.
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove*.zip

--2019-10-28 12:35:10--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2019-10-28 12:35:10--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2019-10-28 12:35:11--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

Some imports lol :P

In [3]:
import pandas as pd
import numpy as np

import torch,keras

from keras.datasets import imdb
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from torch.nn.parameter import Parameter
from torch.nn import init
from torch.autograd import Variable
import torch.nn.functional as F
import torch.utils.data as data_utils

import math

Using TensorFlow backend.


lets load this shit :^)

In [0]:
train_dataframe = pd.read_json('./snli_1.0/snli_1.0_train.jsonl', lines=True)
test_dataframe = pd.read_json('./snli_1.0/snli_1.0_test.jsonl', lines=True)


In [5]:
train_dataframe.head(50)

,annotator_labels,captionID,gold_label,pairID,sentence1,sentence1_binary_parse,sentence1_parse,sentence2,sentence2_binary_parse,sentence2_parse
0,[neutral],3416050480.jpg#4,neutral,3416050480.jpg#4r1n,A person on a horse jumps over a broken down a...,( ( ( A person ) ( on ( a horse ) ) ) ( ( jump...,(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN o...,A person is training his horse for a competition.,( ( A person ) ( ( is ( ( training ( his horse...,(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) ...
1,[contradiction],3416050480.jpg#4,contradiction,3416050480.jpg#4r1c,A person on a horse jumps over a broken down a...,( ( ( A person ) ( on ( a horse ) ) ) ( ( jump...,(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN o...,"A person is at a diner, ordering an omelette.",( ( A person ) ( ( ( ( is ( at ( a diner ) ) )...,(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) ...
2,[entailment],3416050480.jpg#4,entailment,3416050480.jpg#4r1e,A person on a horse jumps over a broken down a...,( ( ( A person ) ( on ( a horse ) ) ) ( ( jump...,(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN o...,"A person is outdoors, on a horse.","( ( A person ) ( ( ( ( is outdoors ) , ) ( on ...",(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) ...
3,[neutral],2267923837.jpg#2,neutral,2267923837.jpg#2r1n,Children smiling and waving at camera,( Children ( ( ( smiling and ) waving ) ( at c...,(ROOT (NP (S (NP (NNP Children)) (VP (VBG smil...,They are smiling at their parents,( They ( are ( smiling ( at ( their parents ) ...,(ROOT (S (NP (PRP They)) (VP (VBP are) (VP (VB...
4,[entailment],2267923837.jpg#2,entailment,2267923837.jpg#2r1e,Children smiling and waving at camera,( Children ( ( ( smiling and ) waving ) ( at c...,(ROOT (NP (S (NP (NNP Children)) (VP (VBG smil...,There are children present,( There ( ( are children ) present ) ),(ROOT (S (NP (EX There)) (VP (VBP are) (NP (NN...
5,[contradiction],2267923837.jpg#2,contradiction,2267923837.jpg#2r1c,Children smiling and waving at camera,( Children ( ( ( smiling and ) waving ) ( at c...,(ROOT (NP (S (NP (NNP Children)) (VP (VBG smil...,The kids are frowning,( ( The kids ) ( are frowning ) ),(ROOT (S (NP (DT The) (NNS kids)) (VP (VBP are...
6,[contradiction],3691670743.jpg#0,contradiction,3691670743.jpg#0r1c,A boy is jumping on skateboard in the middle o...,( ( A boy ) ( ( is ( ( jumping ( on skateboard...,(ROOT (S (NP (DT A) (NN boy)) (VP (VBZ is) (VP...,The boy skates down the sidewalk.,( ( The boy ) ( ( ( skates down ) ( the sidewa...,(ROOT (S (NP (DT The) (NN boy)) (VP (VBZ skate...
7,[entailment],3691670743.jpg#0,entailment,3691670743.jpg#0r1e,A boy is jumping on skateboard in the middle o...,( ( A boy ) ( ( is ( ( jumping ( on skateboard...,(ROOT (S (NP (DT A) (NN boy)) (VP (VBZ is) (VP...,The boy does a skateboarding trick.,( ( The boy ) ( ( does ( a ( skateboarding tri...,(ROOT (S (NP (DT The) (NN boy)) (VP (VBZ does)...
8,[neutral],3691670743.jpg#0,neutral,3691670743.jpg#0r1n,A boy is jumping on skateboard in the middle o...,( ( A boy ) ( ( is ( ( jumping ( on skateboard...,(ROOT (S (NP (DT A) (NN boy)) (VP (VBZ is) (VP...,The boy is wearing safety equipment.,( ( The boy ) ( ( is ( wearing ( safety equipm...,(ROOT (S (NP (DT The) (NN boy)) (VP (VBZ is) (...
9,[neutral],4804607632.jpg#0,neutral,4804607632.jpg#0r1n,An older man sits with his orange juice at a s...,( ( An ( older man ) ) ( ( ( sits ( with ( ( h...,(ROOT (S (NP (DT An) (JJR older) (NN man)) (VP...,An older man drinks his juice as he waits for ...,( ( An ( older man ) ) ( ( ( drinks ( his juic...,(ROOT (S (NP (DT An) (JJR older) (NN man)) (VP...


Helper functions: something that bulk converts things into lists, and a tokeniser that also pads and numpies things

In [0]:
def convert_to_lists(names_to_lists):
  for key in names_to_lists:
    names_to_lists[key] = names_to_lists[key].tolist()
  return names_to_lists

class Tokeniser:
  def __init__(self, texts, vocab_size, max_len):
    self.t = Tokenizer()
    self.max_len = max_len
    self.t.num_words = vocab_size
    
    full_corpus = []

    for index in texts:
      for text in texts[index]:
        full_corpus.append(text)
    
    self.t.fit_on_texts(full_corpus)

  def full_process(self, text):
    """OK SO: converts a list of strings into a list of numerical sequences
then pads them out so they're all a consistent size
then returns a numpy array of that :) """
    new_sequence = self.t.texts_to_sequences(text)
    padded_sequence = pad_sequences(new_sequence, maxlen=self.max_len, padding ="post")
    return np.array(padded_sequence, dtype=np.float32)

  def do_everything(self, texts):
    for index in texts:
      texts[index] = self.full_process(texts[index])
    self.word_to_id = self.t.word_index
    return texts



  
# Get the embedding matrix using Glove. 
vocab,word2idx = None,{}

def load_glove_embeddings(path, word2idx, embedding_dim):
    """Loading the glove embeddings"""
    vocab_size = len(word2idx) + 1
    print(vocab_size)
    with open(path) as f:
        embeddings = np.zeros((vocab_size, embedding_dim))
        for line in f.readlines():
            values = line.split()
            word = values[0]
            index = word2idx.get(word)
            if index:
                vector = np.array(values[1:], dtype='float32')
                if vector.shape[-1] != embedding_dim:
                    raise Exception('Dimension not matching.')
                embeddings[index] = vector
        return torch.from_numpy(embeddings).float()

#assumption: we're going to only care about classification per text
def generate_indexes(labels):
  return [1 if label == "entailment" else 0 for label in labels]

index_to_label = ["contradiction","neutral","entailment"]


here i set up the tokeniser, and turn everything into a list its a fun cell

In [0]:
MAX_LENGTH = 150
VOCAB_SIZE = 20000
BATCH_SIZE = 512


x_train_lists = convert_to_lists({"premise": train_dataframe["sentence1"], "hypothesis": train_dataframe["sentence2"]})
y_train_list = train_dataframe["gold_label"].tolist()

x_test_lists = convert_to_lists({"premise": test_dataframe["sentence1"], "hypothesis": test_dataframe["sentence2"]})
y_test_list = test_dataframe["gold_label"].tolist()


x_tokeniser = Tokeniser(x_train_lists, VOCAB_SIZE, MAX_LENGTH)



this cell uses the setup tokeniser to SLAP THAT SHIT INTO NUMPY ARRAYS WITH PADDING YEAH BABY
(also tokenises it thats p important)

In [19]:
x_train = x_tokeniser.do_everything(x_train_lists)
x_test = x_tokeniser.do_everything(x_test_lists)
y_train = np.array(generate_indexes(y_train_list), dtype=np.float32)
y_test = np.array(generate_indexes(y_test_list), dtype=np.float32)
glove_embeddings = load_glove_embeddings("glove.6B.50d.txt",x_tokeniser.word_to_id,50)

34370


and here we slap the loaded stuff into a neat tensordataset. this is good because ???

In [20]:
print(glove_embeddings.shape)

torch.Size([34370, 50])


In [0]:

#alright lets tensordataset this boy
train_data = data_utils.TensorDataset(torch.from_numpy(x_train["premise"]).type(torch.LongTensor),
                                      torch.from_numpy(x_train["hypothesis"]).type(torch.LongTensor),
                                      torch.from_numpy(y_train).type(torch.DoubleTensor))
train_loader = data_utils.DataLoader(train_data, batch_size=BATCH_SIZE, drop_last=True)

In [0]:
class TextualEntailmentModel(torch.nn.Module):
  """This class will take in two bits of text and try figure out if one bit of text contradicts the other.
  Its an attempt at implementing the model shown in 'A Structured Self-Attentive Sentence Embedding'! 
  """
  def init_hidden(self):
        hidden_state = Variable(torch.zeros(2,self.batch_size,self.lstm_hidden_size))
        cell_state = Variable(torch.zeros(2,self.batch_size, self.lstm_hidden_size))
        return (hidden_state, cell_state)

  def __init__(self, 
               word_embeddings, 
               lstm_hidden_size, 
               dense_dimension, 
               attention_hops,
               batch_size,
               max_length,
               num_classes):

    super(TextualEntailmentModel, self).__init__()
    # ~PREMISE ZONE~
    #loadin in those word embeddings
    self.premise_embeddings = torch.nn.Embedding(word_embeddings.size(0), word_embeddings.size(1))
    self.premise_embeddings.weight = torch.nn.Parameter(word_embeddings)
    self.premise_embedding_size = word_embeddings.size(1)

    #~~ MODEL STRUCTURE ZONE GET HYPE ~~
    self.premise_LSTM = torch.nn.LSTM(
        input_size = self.premise_embedding_size,
        hidden_size = lstm_hidden_size,
        num_layers=1,
        batch_first=True,
        bidirectional=True
      )
    self.premise_first_linear = torch.nn.Linear(
        in_features= 2*lstm_hidden_size,
        out_features = 2*dense_dimension,
        bias = False
    )
    self.premise_second_linear = torch.nn.Linear(
        in_features = 2*dense_dimension,
        out_features = 2*attention_hops,
        bias = False
    )
    self.premise_weight = Parameter(torch.Tensor(
        batch_size, 
        dense_dimension,
        2*attention_hops
        ))
    init.kaiming_uniform_(self.premise_weight, a=math.sqrt(5)) #initialises the premise weight uniformly
    # ~~ END MODEL STRUCTURE FOR PREMISE ~~
    # ~HYPOTHESIS ZONE~ (maybe make this nicer later idk how tbh)
    self.hypothesis_embeddings = torch.nn.Embedding(word_embeddings.size(0), word_embeddings.size(1))
    self.hypothesis_embeddings.weight = torch.nn.Parameter(word_embeddings)
    self.hypothesis_embedding_size = word_embeddings.size(1)

    #~~ MODEL STRUCTURE ZONE GET HYPE ~~
    self.hypothesis_LSTM = torch.nn.LSTM(
        input_size = self.premise_embedding_size,
        hidden_size = lstm_hidden_size,
        num_layers=1,
        batch_first=True,
        bidirectional=True
      )
    self.hypothesis_first_linear = torch.nn.Linear(
        in_features= lstm_hidden_size,
        out_features = dense_dimension,
        bias = False
    )
    self.hypothesis_tanh = torch.nn.Tanh()
    self.hypothesis_second_linear = torch.nn.Linear(
        in_features = dense_dimension,
        out_features = attention_hops,
        bias = False
    )
    self.hypothesis_weight = Parameter(torch.Tensor(
        batch_size, 
        dense_dimension,
        2*attention_hops
        ))
    init.kaiming_uniform_(self.hypothesis_weight, a=math.sqrt(5))
    # ~~ END MODEL STRUCTURE FOR HYPOTHESIS ~~
    self.mlp_friendship_is_magic = torch.nn.Linear(
        in_features=4*dense_dimension, 
        out_features=100)
    self.final_layer = torch.nn.Linear(100, num_classes)
    self.lstm_hidden_size = lstm_hidden_size
    self.batch_size = batch_size
    self.max_length = max_length
    self.initial_hidden_state = self.init_hidden()
    

    
    #currently writing forward function, attempting to translate the lstm part
  def forward(self, premise, hypothesis):
      #Processing premises maybe?
    premise_word_embeddings = self.premise_embeddings(premise)
    premise_embeddings_batched = premise_word_embeddings.view(
        self.batch_size, 
        self.max_length, 
        -1
    )
    premise_outputs, premise_hidden_state = self.premise_LSTM(
        premise_embeddings_batched, 
        self.initial_hidden_state
    )

    premise_tanh_W_H = torch.tanh(self.premise_first_linear(premise_outputs))
    premise_by_attention_hops = self.premise_second_linear(premise_tanh_W_H)
    premise_attention = self.softmax(premise_by_attention_hops).transpose(1,2)

    premise_sentence_embeddings = premise_attention@premise_outputs
    premise_factor = self.bdot(premise_sentence_embeddings, 
                                 self.premise_weight)
      
    #Processing hypothesese
    hypothesis_word_embeddings = self.hypothesis_embeddings(hypothesis)

    hypothesis_embeddings_batched = hypothesis_word_embeddings.view(
        self.batch_size, 
        self.max_length, 
        -1
    )
    hypothesis_outputs, hypothesis_hidden_state = self.hypothesis_LSTM(
        hypothesis_embeddings_batched,
        premise_hidden_state
    )

    hypothesis_tanh_W_H = torch.tanh(self.premise_first_linear(hypothesis_outputs))
    hypothesis_by_attention_hops = self.premise_second_linear(hypothesis_tanh_W_H)
    hypothesis_attention = self.softmax(hypothesis_by_attention_hops).transpose(1,2)
    hypothesis_sentence_embeddings = hypothesis_attention@hypothesis_outputs

    hypothesis_factor = self.bdot(hypothesis_sentence_embeddings,
                                  self.hypothesis_weight)

      # ok lets do the rest

    mush = premise_factor * hypothesis_factor
    almost_output = self.mlp_friendship_is_magic(mush.view(self.batch_size, -1))
    output = self.softmax(self.final_layer(almost_output))
    return output

      



  def bdot(self, a, b):
    B = a.shape[0]
    S = a.shape[1]
    return torch.bmm(a, b)




  def softmax(self,input, axis=1):
    """
        Softmax applied to axis=n
 
        Args:
           input: {Tensor,Variable} input on which softmax is to be applied
           axis : {int} axis on which softmax is to be applied
 
        Returns:
            softmaxed tensors
 
       
    """
    input_size = input.size()
    trans_input = input.transpose(axis, len(input_size)-1)
    trans_size = trans_input.size()
    input_2d = trans_input.contiguous().view(-1, trans_size[-1])
    soft_max_2d = F.softmax(input_2d)
    soft_max_nd = soft_max_2d.view(*trans_size)  
    return soft_max_nd.transpose(axis, len(input_size)-1)

    




In [0]:
def train(model=None, 
          train_loader=None, 
          loss_function=None, 
          optimiser=None, 
          epochs=5, 
          using_gradient_clipping=False):
  
  losses = []
  accuracies = []
  for epoch in range(epochs):
    print("Running EPOCH:",epoch+1)
    total_loss = 0
    batch_count = 0
    correct = 0

    for batch_index, train_data in enumerate(train_loader):
      model.hidden_state = model.init_hidden()
      premise = Variable(train_data[0])
      hypothesis = Variable(train_data[1])
      actual_y = Variable(train_data[2])
      
      predicted_y = model(premise, hypothesis)
      squeezed_y = predicted_y.double().squeeze(1)
      loss = loss_function(squeezed_y,actual_y)

      correct += torch.eq(torch.round(squeezed_y), actual_y).data.sum()
      
      total_loss += loss.data

      #woah we gotta do this to do backprop!!!
      optimiser.zero_grad()
      loss.backward()

      if using_gradient_clipping:
        torch.nn.utils.clip_grad_norm(model.parameters(), 0.5)
      batch_count += 1
      optimiser.step()

    print("Average loss is:",total_loss/batch_count)
    correct_but_numpy = correct.data.cpu().numpy().astype(int)
    accuracy = correct_but_numpy / float(batch_count * train_loader.batch_size)
    print("Accuracy of the model", accuracy)
    losses.append(total_loss/batch_count)
    accuracies.append(accuracy)
  return losses, accuracies

def evaluate(model, test_premise, test_hypothesis, test_y):

  model.batch_size = test_hypothesis.shape[0] #why
  model.hidden_state = model.init_hidden()

  premise_variable = Variable(torch.from_numpy(test_premise).type(torch.LongTensor))
  hypothesis_variable = Variable(torch.from_numpy(test_hypothesis).type(torch.LongTensor))
  y_actual_variable = Variable(torch.from_numpy(test_y).type(torch.DoubleTensor))
  y_predicted = model(test_premise, test_hypothesis)
  y_predicted_rounded = torch.round(y_predicted.type(torch.DoubleTensor).squeeze(1))

  test_data_count = premise_variable.size(0)



  total_accuracy = torch.eq(y_predicted, y_actual_variable).data.sum()
  average_accuracy = total_accuracy.data.cpu().numpy().astype(int)/float(test_data_count)
  return average_accuracy, y_predicted_rounded



      

OK WE MADE THE HELPERS LETS RUN THIS SHIT :)

In [36]:
lstm_hidden_dimension = 50
dense_dimension = 100
amount_of_hops = 10
textual_entailment_model = TextualEntailmentModel(batch_size=train_loader.batch_size,
                               max_length = MAX_LENGTH,
                               lstm_hidden_size = lstm_hidden_dimension,
                               dense_dimension = dense_dimension,
                               attention_hops = amount_of_hops,
                               word_embeddings = glove_embeddings,
                               num_classes = 1)

cross_entropy_loss = torch.nn.BCELoss()
rms_optimiser = torch.optim.RMSprop(textual_entailment_model.parameters(), weight_decay=)
loss, accuracy = train(model=textual_entailment_model,
                       train_loader=train_loader,
                       loss_function=cross_entropy_loss,
                       optimiser = rms_optimiser,
                       epochs = 5,
                       using_gradient_clipping=True
                      )


Running EPOCH: 1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:172: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:35: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.


Average loss is: tensor(18.4190, dtype=torch.float64)
Accuracy of the model 0.33339334555400374
Running EPOCH: 2
Average loss is: tensor(18.4190, dtype=torch.float64)
Accuracy of the model 0.33339334555400374
Running EPOCH: 3
Average loss is: tensor(18.4190, dtype=torch.float64)
Accuracy of the model 0.33339334555400374
Running EPOCH: 4


KeyboardInterrupt: ignored